# Alpha Factor Evaluation

### Loading libraries

In [17]:
# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd
import pandas_datareader.data as web

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Model Interpreter
import shap

# Warnings
import warnings

# Path, OS & Time
import os, sys
from time import time
from pathlib import Path

# StatsModel
import statsmodels.api as sm

# Scikit-Learn
from sklearn.preprocessing import scale
from sklearn.feature_selection import mutual_info_regression

# SciPy
from scipy.stats import spearmanr

# Gradient Booster
import lightgbm as lgb

In [21]:
from utils import MultipleTimeSeriesCV

In [25]:
sys.path.insert(1, os.path.join(sys.path[0], '..'))

In [27]:
idx= pd.IndexSlice

sns.set_style('whitegrid')

warnings.filterwarnings('ignore')

In [29]:
results_path = Path('results')

if not results_path.exists():
    results_path.mkdir()

### Loading Data